<a href="https://colab.research.google.com/github/pshreyareddy/CourseProject/blob/main/ProjectCodeForBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sarcasm Classification In Tweets



In [ ]:
# Install TensorFlow packages
!pip install tensorflow==2.1.0
!pip install tensorflow-hub
!pip install tensorflow-addons
!pip install sentencepiece

     |████████████████████████████████| 421.8MB 29kB/s 
     |████████████████████████████████| 450kB 44.9MB/s 
     |████████████████████████████████| 3.9MB 50.6MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=87b731db30a50d1d1a5186c219e7347a0ba0450d3b5a003a066df6b15bd3a936
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninsta

Import Required Packages 

In [ ]:
import tensorflow as tf; 

print(tf.__version__)

2.1.0


In [ ]:
!python --version

Python 3.6.9


In [ ]:
import pandas as pd
import numpy as np
import re
import string
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Import tensorflow packages
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,concatenate,Dense,GlobalAveragePooling1D,Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_addons as tfa


Read Train Data Set

In [ ]:

train = pd.read_json('https://raw.githubusercontent.com/CS410Fall2020/ClassificationCompetition/main/data/train.jsonl',lines=True)


Check train data 

In [ ]:
train.head()

,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [ ]:
train.dtypes

label       object
response    object
context     object
dtype: object

Clean the response column in train dataset

In [ ]:
train['response']=train['response'].str.replace('@USER', "") 
train['response']=train['response'].str.replace('<URL>', "") 
train['response']=train['response'].str.replace('\d+', '')
train['response']=train['response'].str.lower()
train['response']=train['response'].str.replace('[^\w\s]','')

Check the train data after changes

In [ ]:
train['response'][0]

'   i dont get this  obviously you do care or you wouldve moved right along  instead you decided to care and troll her '

In [ ]:
train['response'].head()

0       i dont get this  obviously you do care or y...
1      trying to protest about  talking about him a...
2       he makes an insane about of money from the ...
3      meanwhile trump wont even release his sat sc...
4      pretty sure the antilincoln crowd claimed th...
Name: response, dtype: object

Convert context(list) into a string and apply the same cleaning logic added on response column

In [ ]:
train['context']=train['context'].apply(lambda x: ','.join(map(str, x)))
train['context']=train['context'].str.replace('@USER', "") 
train['context']=train['context'].str.replace('<URL', "") 
train['context']=train['context'].str.lower()
train['context']=train['context'].str.replace('[^\w\s]','')
train['context']=train['context'].str.replace('\d+', '')


Check the train data after changes

In [ ]:
train['context'][0]

'a minor child deserves privacy and should be kept out of politics  pamela karlan  you should be ashamed of your very angry and obviously biased public pandering  and using a child to do it  if your child isnt named barron  bebest melania couldnt care less  fact  '

In [ ]:
train['context'].head()

0    a minor child deserves privacy and should be k...
1      why is he a loser  hes just a press secretar...
2    donald j  trump is guilty as charged  the evid...
3    jamie raskin tanked doug collins  collins look...
4    man  y  all gone  both sides  the apocalypse o...
Name: context, dtype: object

Check the length of train data set

In [ ]:
len(train)

5000

Check the class count of target variable 'label' (Balanced dataset with equal composition)

In [ ]:
from collections import Counter
Counter(train['label'])

Counter({'NOT_SARCASM': 2500, 'SARCASM': 2500})

Label encode class variable to 0 and 1 from  NOT_SARCASM and SARCASM

In [ ]:
le=LabelEncoder()
train_label=le.fit_transform(train['label'])

Read test data set and check the first 5 rows 

In [ ]:

test = pd.read_json('https://raw.githubusercontent.com/CS410Fall2020/ClassificationCompetition/main/data/test.jsonl',lines=True)

test.head()

,id,response,context
0,twitter_1,"@USER @USER @USER My 3 year old , that just fi...","[Well now that ’ s problematic AF <URL>, @USER..."
1,twitter_2,@USER @USER How many verifiable lies has he to...,[Last week the Fake News said that a section o...
2,twitter_3,@USER @USER @USER Maybe Docs just a scrub of a...,[@USER Let ’ s Aplaud Brett When he deserves i...
3,twitter_4,@USER @USER is just a cover up for the real ha...,[Women generally hate this president . What's ...
4,twitter_5,@USER @USER @USER The irony being that he even...,"[Dear media Remoaners , you excitedly sharing ..."


In [ ]:
test.dtypes

id          object
response    object
context     object
dtype: object

Apply the same cleaning logic for response and context columns in test  as done earlier on train data set

In [ ]:
test['response']=test['response'].str.replace('@USER', "") 
test['response']=test['response'].str.replace('<URL>', "") 
test['response']=test['response'].str.replace('\d+', '')
test['response']=test['response'].str.lower()
test['response']=test['response'].str.replace('[^\w\s]','')
test['context']=test['context'].apply(lambda x: ','.join(map(str, x)))
test['context']=test['context'].str.replace('@USER', "") 
test['context']=test['context'].str.replace('<URL>', "") 
test['context']=test['context'].str.lower()
test['context']=test['context'].str.replace('[^\w\s]','')
test['context']=test['context'].str.replace('\d+', '')


Check the test rows after changes

In [ ]:
test['response'][0]

'   my  year old  that just finished reading nietzsche and then asked me   ayo papa why these people always trying to cancel someone on twitter  trying to pretend like that makes them better themselves    to which i replied  idk   and he just  cuz hoes mad   im so proud  '

In [ ]:
test['context'][0]

'well now that  s problematic af   my  year old  asked me why they are making fun of native americans    i will take shit that didnt happen for     no  he actually in the gifted program and reads on second grade level    and he knows kansas city is in missouri'

In [ ]:
test.head()

,id,response,context
0,twitter_1,my year old that just finished reading ni...,well now that s problematic af my year old...
1,twitter_2,how many verifiable lies has he told now d...,last week the fake news said that a section of...
2,twitter_3,maybe docs just a scrub of a coach i mean ...,let s aplaud brett when he deserves it he co...
3,twitter_4,is just a cover up for the real hate inside ...,women generally hate this president whats up ...
4,twitter_5,the irony being that he even has to ask why,dear media remoaners you excitedly sharing cl...


Check the train rows after changes 

In [ ]:
train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,SARCASM,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,SARCASM,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,SARCASM,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


Create a variable traindata from train 

In [ ]:
traindata = train

In [ ]:
len(train_label)

5000

Train Validation Split (To train on 4000 samples and validate on 1000 samples)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(traindata.drop(labels=['label'], axis=1), train_label, test_size=0.2, random_state=42 ,stratify=train_label)
#


In [ ]:
X_train.shape   #Check the shape of X_train

(4000, 2)

In [ ]:
y_train.shape  #Check the shape of y_train


(4000,)

In [ ]:
 X_val.shape   #Check the shape of X_val

(1000, 2)

Check the target variable balance in training and validation 

In [ ]:
print(Counter(y_val))  
print(Counter(y_train))

Counter({0: 500, 1: 500})
Counter({1: 2000, 0: 2000})


Added a keras bert layer using bert uncased L-12_H-768_A-12. Tokenization approach using bert fulltokenizer followed as per usage in below tfhub link

https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1

In [ ]:
import tensorflow_hub as hub
hub

<module 'tensorflow_hub' from '/usr/local/lib/python3.6/dist-packages/tensorflow_hub/__init__.py'>

In [ ]:

module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)


Download and import bert tokenization file

In [ ]:
#!wget --quiet https://raw.githubusercontent.com/google-research/BERT/master/tokenization.py
#!cp "/content/drive/MyDrive/tokenization.py"
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
import tokenization
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

Encoding function to separate the text into tokens,masks and segments

In [ ]:
def encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
 max_len=256

In [ ]:
val_response = encode(X_val.response, tokenizer, max_len=max_len)
val_context = encode(X_val.context, tokenizer, max_len=max_len)
val_labels=y_val

In [ ]:
train_response = encode(X_train.response, tokenizer, max_len=max_len)
train_context = encode(X_train.context, tokenizer, max_len=max_len)
train_labels=y_train


In [ ]:
test_response = encode(test.response, tokenizer, max_len=max_len)
test_context = encode(test.context, tokenizer, max_len=max_len)

In [ ]:
test_input=[test_context[0],test_context[1],test_context[2],test_response[0],test_response[1],test_response[2]]

Helper functions for Evaluation metrics like recall,precision and f1 score

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Build model


In [ ]:
def build_model(bert_layer, max_len=512):
    context_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="contex_word_ids")
    context_mask = Input(shape=(max_len,), dtype=tf.int32, name="context_mask")
    context_seg_ids = Input(shape=(max_len,), dtype=tf.int32, name="context_seg_ids")
    context_pooled_output , context_seq_output = bert_layer([context_word_ids, context_mask, context_seg_ids])
    context_clf_output = context_seq_output[:, 0, :]


    reponse_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_word_ids")
    reponse_mask = Input(shape=(max_len,), dtype=tf.int32, name="reponse_mask")
    reponse_seg_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_seg_ids")
    response_pooled_output , reponse_seq_output = bert_layer([reponse_word_ids, reponse_mask, reponse_seg_ids])
    reponse_clf_output = reponse_seq_output[:, 0, :]


    conclayer=concatenate([context_clf_output,reponse_clf_output])
    out = Dense(1, activation='sigmoid')(conclayer)
    
    model = Model(inputs=[context_word_ids, context_mask, context_seg_ids,reponse_word_ids,reponse_mask,reponse_seg_ids], outputs=out)
    model.compile(Adam(1e-6), loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

    return model

# context_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="contex_word_ids")
# context_mask = Input(shape=(max_len,), dtype=tf.int32, name="context_mask")
# context_seg_ids = Input(shape=(max_len,), dtype=tf.int32, name="context_seg_ids")
# context_pooled_output , context_seq_output = bert_layer([context_word_ids, context_mask, context_seg_ids])
# context_clf_output = context_seq_output[:, 0, :]


# reponse_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_word_ids")
# reponse_mask = Input(shape=(max_len,), dtype=tf.int32, name="reponse_mask")
# reponse_seg_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_seg_ids")
# response_pooled_output , reponse_seq_output = bert_layer([reponse_word_ids, reponse_mask, reponse_seg_ids])
# reponse_clf_output = reponse_seq_output[:, 0, :]


# conclayer=concatenate([context_clf_output,reponse_clf_output])
# out = Dense(1, activation='sigmoid')(conclayer)
    
# model = Model(inputs=[context_word_ids, context_mask, context_seg_ids,reponse_word_ids,reponse_mask,reponse_seg_ids], outputs=out)
# model.compile(Adam(1e-6), loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])


In [ ]:

model = build_model(bert_layer, max_len=max_len)
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
contex_word_ids (InputLayer)    [(None, 256)]        0                                            
__________________________________________________________________________________________________
context_mask (InputLayer)       [(None, 256)]        0                                            
__________________________________________________________________________________________________
context_seg_ids (InputLayer)    [(None, 256)]        0                                            
__________________________________________________________________________________________________
reponse_word_ids (InputLayer)   [(None, 256)]        0                                            
____________________________________________________________________________________________

Set train_input

In [ ]:
train_input=[train_context[0],train_context[1],train_context[2],train_response[0],train_response[1],train_response[2]]

Set val_input

In [ ]:
val_input=[val_context[0],val_context[1],val_context[2],val_response[0],val_response[1],val_response[2]]

Training the model 

In [ ]:
train_history = model.fit(
    train_input, y_train,
    validation_data =(val_input,y_val),
    epochs=3,
    batch_size=3)

Train on 4000 samples, validate on 1000 samples
Epoch 1/3
4000/4000 [==============================] - 337s 84ms/sample - loss: 0.5813 - acc: 0.6815 - f1_m: 0.5717 - precision_m: 0.5910 - recall_m: 0.6128 - val_loss: 0.4926 - val_acc: 0.7600 - val_f1_m: 0.6401 - val_precision_m: 0.6572 - val_recall_m: 0.6781
Epoch 2/3
4000/4000 [==============================] - 314s 79ms/sample - loss: 0.4332 - acc: 0.7987 - f1_m: 0.6909 - precision_m: 0.6939 - recall_m: 0.7348 - val_loss: 0.4491 - val_acc: 0.7880 - val_f1_m: 0.6716 - val_precision_m: 0.6712 - val_recall_m: 0.7231
Epoch 3/3
4000/4000 [==============================] - 314s 79ms/sample - loss: 0.3316 - acc: 0.8685 - f1_m: 0.7633 - precision_m: 0.7645 - recall_m: 0.7969 - val_loss: 0.4332 - val_acc: 0.7970 - val_f1_m: 0.6594 - val_precision_m: 0.6722 - val_recall_m: 0.6946


Predictions for test_input for the trained model

In [ ]:
pred=model.predict(test_input,verbose=1)

1800/1800 [==============================] - 39s 22ms/sample


Write the predictions for test dataset into answer.txt with columns id and label

In [ ]:

sub=pred.round()
sub=le.inverse_transform(sub.ravel().astype('int16'))
sub=pd.DataFrame(sub,columns=['label'])
sub=pd.concat([test['id'], sub], axis=1)
sub.head()
sub.to_csv('answer.txt',sep=',',index=False,header=None)

Save model weights

In [ ]:
model_save_name = 'bert_weights1.h5'
path = F"/content/drive/My Drive/{model_save_name}" 
model.save_weights(path)

Loading model weights from already generated saved model weights file


In [ ]:
model2 = build_model(bert_layer, max_len=max_len)
model2.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
contex_word_ids (InputLayer)    [(None, 256)]        0                                            
__________________________________________________________________________________________________
context_mask (InputLayer)       [(None, 256)]        0                                            
__________________________________________________________________________________________________
context_seg_ids (InputLayer)    [(None, 256)]        0                                            
__________________________________________________________________________________________________
reponse_word_ids (InputLayer)   [(None, 256)]        0                                            
____________________________________________________________________________________________

In [ ]:
print('load model')
# model2.load_weights('/content/drive/My Drive/bert_weights.h5')

# https://drive.google.com/file/d/1hWyGrsE69dCJBK4HI-VFJql7PDiEG2LA/edit

#https://drive.google.com/file/d/1bHc2hz5rJ7eyYIZqPtBhgVXMy1fsMCc_/view?usp=sharing

#!gdown --id 1hWyGrsE69dCJBK4HI-VFJql7PDiEG2LA

!gdown --id 1bHc2hz5rJ7eyYIZqPtBhgVXMy1fsMCc_





load model
Downloading...
From: https://drive.google.com/uc?id=1bHc2hz5rJ7eyYIZqPtBhgVXMy1fsMCc_
To: /content/bert_weights1.h5
438MB [00:01, 247MB/s]


In [ ]:
model2.load_weights('bert_weights1.h5')

In [ ]:
pred2 = model2.predict(test_input,verbose=1)



1800/1800 [==============================] - 38s 21ms/sample


In [ ]:
sub=pred2.round()
sub=le.inverse_transform(sub.ravel().astype('int16'))
sub=pd.DataFrame(sub,columns=['label'])
sub=pd.concat([test['id'], sub], axis=1)
sub.head()
sub.to_csv('answer.txt',sep=',',index=False,header=None)